In [1]:
# Imports
import numpy as np

In [2]:
# Import the tools
from sys import path
path.append('/Users/reid/dev/PythonCode/tm/tools')
path.append('D:/PythonCode/tm/tools')
from tools import gen_inputs_outputs, logsig, lin, deriv_logsig, deriv_lin, e2

In [3]:
# Define the size of the network
# Order by layers (input, layer1, layer2, ...)
# !!! No output layer yet, wait to gen inputs
layer_sizes = [4, 7]

In [4]:
# Gen inputs, outputs and size of the last layer
inputs, outputs, S = gen_inputs_outputs(layer_sizes[0])

In [5]:
# Append the output layer size
layer_sizes = np.array(layer_sizes + [S], dtype=int)

In [6]:
# Network shape
layer_sizes

array([4, 7, 3])

In [7]:
# Inputs
inputs

array([[0, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 1, 0],
       [0, 0, 1, 1],
       [0, 1, 0, 0],
       [0, 1, 0, 1],
       [0, 1, 1, 0],
       [0, 1, 1, 1],
       [1, 0, 0, 0],
       [1, 0, 0, 1],
       [1, 0, 1, 0],
       [1, 0, 1, 1],
       [1, 1, 0, 0],
       [1, 1, 0, 1],
       [1, 1, 1, 0],
       [1, 1, 1, 1]])

In [8]:
# Outputs
outputs

array([[0, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 1],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 1],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 1],
       [1, 0, 0],
       [1, 0, 1],
       [0, 1, 1],
       [1, 0, 0],
       [1, 0, 1],
       [1, 1, 0]])

In [9]:
# Intialize the weights and biases randomly
# Also create the list to store the outputs of each layer
# Store the outptut for each layer
# Finally create the list of the sensitivities
weights_list = []
biases_list = []
n_list = []
a_list = []
s_list = []

In [10]:
# Define a function to reset all the parameters
# This is done to test multiple different network types
def reset_params():
    weights_list.clear()
    biases_list.clear()
    n_list.clear()
    a_list.clear()
    s_list.clear()
    for i in range(1, len(layer_sizes)):
        weights_list.append(np.random.rand(layer_sizes[i], layer_sizes[i-1]))
        biases_list.append(np.random.rand(layer_sizes[i], 1))
        n_list.append(np.empty((layer_sizes[i], 1), dtype=float))
        a_list.append(np.empty((layer_sizes[i], 1), dtype=float))
        s_list.append(np.empty((layer_sizes[i], 1), dtype=float))

In [11]:
# Define the transfer functions
transfer_functions = [logsig, lin]

In [12]:
# Define the vectors with the derivatives of the transfer functions
# These need to be converted to dialation matrices
# but this is done after the partial derivatives are calculated
# so that the numerical values can be multiplied by the identity matrix
# makes my life easier
deriv_transfer_functions = [deriv_logsig, deriv_lin]

In [13]:
# Define a function to calculate the output of the network
# while also saving the data
# Assuming that the input is already a Rx1 matrix
def run_network(X):
    # Get the input for the first layer so that
    # no need to use X and can calculate
    # recursively
    n_list[0] = np.matmul(weights_list[0], X) + biases_list[0]
    a_list[0] = transfer_functions[0](n_list[0])

    # Calculate the rest of the outptut
    for i in range(1, len(weights_list)):
        n_list[i] = np.matmul(weights_list[i], a_list[i-1]) + biases_list[i]
        a_list[i] = transfer_functions[i](n_list[i])

In [14]:
# Calculate the senstivites for the last layer
# Write a function to fill the diagonal of the matrix
def fill_F_dot(layer_num):
    a = np.identity(len(n_list[layer_num]), float)
    np.fill_diagonal(a, deriv_transfer_functions[layer_num](n_list[layer_num]).flatten())
    return a

def train_network(learning_rate=.1, epochs=1):
    for i in range(epochs):
        # Iterate through the inputs and outputs
        for i in range(len(inputs)):
            # Run the network
            run_network(inputs[i].reshape(layer_sizes[0], 1))

            # Get the dialation matrix for the last layer
            F_dot = fill_F_dot(-1)

            # Calculate the sensitivites for the last layer
            s_list[-1] = -2 * np.matmul(F_dot, outputs[i].reshape(layer_sizes[-1], 1)-a_list[-1])

            # Update the weights and biases for the last layer
            weights_list[-1] += -learning_rate*np.matmul(s_list[-1], a_list[-2].T)
            biases_list[-1] += -learning_rate*s_list[-1]

            # Iterate through the remaining layers backwards
            for i in range(len(n_list)-2, 0, -1):
                s_list[i] = (1/layer_sizes[i+1]) * np.matmul(np.matmul(fill_F_dot(i), weights_list[i+1].T), s_list[i+1])
                weights_list[i] += -learning_rate*np.matmul(s_list[i], a_list[i-1].T)
                biases_list[i] += -learning_rate*s_list[i]

In [15]:
# Function to evaluate the performance
# of the network
def performance():
    results = np.empty((len(inputs), layer_sizes[-1]), dtype=float)
    for i in range(len(inputs)):
        run_network(inputs[i].reshape(layer_sizes[0], 1))
        results[i] = outputs[i] - a_list[-1].flatten()
    return e2(results)

In [16]:
# Print performance
def print_performance_dict(performance_dict):
    for a in sorted(performance_dict.items(), key=lambda x: x[1]):
        print(f'Error: {a[1]} | ' + a[0])

In [17]:
# Trains netowrk using the parameters for each bit size
def update_performance_dict(layer_sizes_list, performance_dict):
    for layer_sizes in layer_sizes_list:
        # Get the inputs and outputs for the specifiq network size
        inputs, outputs, S = gen_inputs_outputs(layer_sizes[0])
        layer_sizes.append(S)
    
        # Iterate through the different learning rates
        # and epochs
        for rate in learning_rates:
            for epochs in epochs_list:
                # Reset weights and biases
                reset_params()
                # Train the network
                train_network(learning_rate=rate, epochs=epochs)
                # Update the performance dict
                performance_dict.update({f'Layers: {layer_sizes}, Learning rate: {rate}, Epochs: {epochs}': performance()})

In [18]:
### !!! Do not run code underneath until saving to the file
### It trains 22000 different networks
# Performance dictionnaries of the network
performance_dict_2bits = {}
performance_dict_4bits = {}
performance_dict_8bits = {}

In [19]:
# Parameters that won't change despite differnt layer sizes
# Set different learning rates
learning_rates = [1/(16*(2**i)) for i in range(10)]

# Set the diffrent epochs
epochs_list = [i for i in range(1, 21)]

In [20]:
# Multiple different layer sizes
# Start with 1 hidden layer
layer_sizes_list_2bits = [[4, 2*(2**i)]for i in range(10)]
layer_sizes_list_4bits = [[8, 2*(2**i)]for i in range(10)]
layer_sizes_list_8bits = [[16, 2*(2**i)]for i in range(10)]

In [21]:
# Calculate performance for 1 hidden layer
# 2 bits
update_performance_dict(layer_sizes_list_2bits, performance_dict_2bits)
# 4 bits
update_performance_dict(layer_sizes_list_4bits, performance_dict_4bits)
# 8 bits
update_performance_dict(layer_sizes_list_8bits, performance_dict_8bits)

In [22]:
# 2 Hidden layers
# Redefine the transfer functions
transfer_functions = [logsig, logsig, lin]
deriv_transfer_functions = [deriv_logsig, deriv_logsig, deriv_lin]

# New layer sizes
layer_sizes_list_2bits = [[4, 2*(2**i), 2*(2**j)] for j in range(10) for i in range(10)]
layer_sizes_list_4bits = [[8, 2*(2**i), 2*(2**j)] for j in range(10) for i in range(10)]
layer_sizes_list_8bits = [[16, 2*(2**i), 2*(2**j)] for j in range(10) for i in range(10)]

In [23]:
# Calculate the performance for 2 hidden layers
# 2 bits
update_performance_dict(layer_sizes_list_2bits, performance_dict_2bits)
# 4 bits
update_performance_dict(layer_sizes_list_4bits, performance_dict_4bits)
# 8 bits
update_performance_dict(layer_sizes_list_8bits, performance_dict_8bits)

In [24]:
print('-'*7 + '2bits' + '-'*7)
print_performance_dict(performance_dict_2bits)
print('-'*7 + '4bits' + '-'*7)
print_performance_dict(performance_dict_4bits)
print('-'*7 + '8bits' + '-'*7)
print_performance_dict(performance_dict_8bits)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [25]:
# Save the data of the performance dicts
performance_dicts = [performance_dict_2bits, performance_dict_4bits, performance_dict_8bits]
for i in range(len(performance_dicts)):
    with open(f'/Users/reid/dev/PythonCode/tm/BackPropagationBinaryCalcs/data/{2**(i+1)}bit_performance.txt', 'w') as f:
        f.write('parameters,performance\n')
        f.writelines([f'{x}\n' for x in sorted(list(performance_dicts[i].items()), key=lambda x: x[1], reverse=True)])
### You can run code below this

In [18]:
# Chose a few networks from the data file to run tests on
# General parameters
learning_rates = [0.001953125, 0.00390625, 0.0078125]
epochs_list = [12, 15, 17, 5, 14, 19]

In [19]:
# Start with one hidden layer
transfer_functions = [logsig, lin]
deriv_transfer_functions = [deriv_logsig, deriv_lin]

layer_sizes_list_2bits = [
    [4, 2],
    [4, 32],
    [4, 64],
]
layer_sizes_list_4bits = [
    [8, 64],
    [8, 2],
    [8, 1024],
]
layer_sizes_list_8bits = [
    [16, 16],
    [16, 64],
    [16, 4],
]

In [20]:
# Write a function to calculate the average performance
def update_average_performance_dict(layer_sizes_list, average_performance_dict, n_samples):
    for layer_sizes in layer_sizes_list:
        inputs, outputs, S = gen_inputs_outputs(layer_sizes[0])
        layer_sizes.append(S)

        cumulated_performance = 0
        
        for rate in learning_rates:
                for epochs in epochs_list:
                    for i in range(n_samples):
                        reset_params()
                        train_network(learning_rate=rate, epochs=epochs)
                        cumulated_performance += performance()
                        
                    average_performance_dict.update({f'Layers: {layer_sizes}, Learning rate: {rate}, Epochs: {epochs}': cumulated_performance/n_samples})   

In [21]:
average_performance_dict_2bits = {}
average_performance_dict_4bits = {}
average_performance_dict_8bits = {}

In [22]:
update_average_performance_dict(layer_sizes_list_2bits, average_performance_dict_2bits, 500)
update_average_performance_dict(layer_sizes_list_4bits, average_performance_dict_4bits, 500)
update_average_performance_dict(layer_sizes_list_8bits, average_performance_dict_8bits, 500)

In [23]:
# Two hidden layers
transfer_functions = [logsig, logsig, lin]
deriv_transfer_functions = [deriv_logsig, deriv_logsig, deriv_lin]

layer_sizes_list_2bits = [
    [4, 8, 2],
    [4, 2, 128],
    [4, 256, 32],
]
layer_sizes_list_4bits = [
    [8, 32, 64],
    [8, 1024, 64],
    [8, 2, 256],
]
layer_sizes_list_8bits = [
    [16, 8, 8],
    [16, 2, 1024],
    [16, 16, 4],
]

In [24]:
update_average_performance_dict(layer_sizes_list_2bits, average_performance_dict_2bits, 500)
update_average_performance_dict(layer_sizes_list_4bits, average_performance_dict_4bits, 500)
update_average_performance_dict(layer_sizes_list_8bits, average_performance_dict_8bits, 500)

In [25]:
print('----------2 bit average----------')
print_performance_dict(average_performance_dict_2bits)
print('----------4 bit average----------')
print_performance_dict(average_performance_dict_4bits)
print('----------8 bit average----------')
print_performance_dict(average_performance_dict_8bits)

----------2 bit average----------
Error: 11.479299935406559 | Layers: [4, 2, 3], Learning rate: 0.001953125, Epochs: 12
Error: 11.53308861576714 | Layers: [4, 64, 3], Learning rate: 0.001953125, Epochs: 12
Error: 11.542363713634707 | Layers: [4, 32, 3], Learning rate: 0.001953125, Epochs: 12
Error: 17.803159669594926 | Layers: [4, 256, 32, 3], Learning rate: 0.001953125, Epochs: 12
Error: 17.81829355607529 | Layers: [4, 8, 2, 3], Learning rate: 0.001953125, Epochs: 12
Error: 17.8565476440209 | Layers: [4, 2, 128, 3], Learning rate: 0.001953125, Epochs: 12
Error: 22.912720139622905 | Layers: [4, 2, 3], Learning rate: 0.001953125, Epochs: 15
Error: 22.932651544335705 | Layers: [4, 32, 3], Learning rate: 0.001953125, Epochs: 15
Error: 22.959130223979894 | Layers: [4, 64, 3], Learning rate: 0.001953125, Epochs: 15
Error: 34.038750350235546 | Layers: [4, 256, 32, 3], Learning rate: 0.001953125, Epochs: 15
Error: 34.04837897686988 | Layers: [4, 8, 2, 3], Learning rate: 0.001953125, Epochs: 1

In [27]:
# Save the average performance data
performance_dicts = [average_performance_dict_2bits, average_performance_dict_4bits, average_performance_dict_8bits]
for i in range(len(performance_dicts)):
    with open(f'./data/{2**(i+1)}bit_average_performance.txt', 'w') as f:
        f.write('parameters,performance\n')
        f.writelines([f'{x}\n' for x in sorted(list(performance_dicts[i].items()), key=lambda x: x[1], reverse=True)])

In [4]:
# Check whether the average data goes by steps for each network
from csv import reader

with open ('./data/8bit_average_performance.txt', 'r') as f:
    csv_reader = reader(f, delimiter=',')
    network_params =  []
    
    skip = True
    for row in csv_reader:
        if skip: skip = False; continue
        
        if len(row)==6:
            network_params.append({'layers': row[0]+row[1]+row[2], 'lr': row[3], 'epochs': row[4], 'error': float(row[5][1:-1])})
        
        if len(row)==7:
            network_params.append({'layers': row[0]+row[1]+row[2]+row[3], 'lr': row[4], 'epochs': row[5], 'error': float(row[6][1:-1])})
    
    for i in range(0, len(network_params), 3):
        layers_steps = [network_params[i]['layers'], 
                       network_params[i+1]['layers'], 
                       network_params[i+2]['layers']]
        
        epochs_steps = [network_params[i]['epochs'], 
                       network_params[i+1]['epochs'], 
                       network_params[i+2]['epochs']]
        
        lr_steps = [network_params[i]['lr'], 
                       network_params[i+1]['lr'], 
                       network_params[i+2]['lr']]
        
        err_steps = [round(network_params[i]['error']), 
                       round(network_params[i+1]['error']), 
                       round(network_params[i+2]['error'])]
        
        print(f'Step from index {i}')
        
        if len(layers_steps) == len(set(layers_steps)):
            print('All layers in step the unique')
        
        if 1 == len(set(epochs_steps)):
            print('All epochs in step the same')
            
        if 1 == len(set(lr_steps)):
            print('All lr in step the same')
        
        if 1 == len(set(err_steps)):
            print('All err within round')
        else: print(set(err_steps))
            
        print('-----------------------------------')

Step from index 0
All layers in step the unique
All epochs in step the same
All lr in step the same
All err within round
-----------------------------------
Step from index 3
All layers in step the unique
All epochs in step the same
All lr in step the same
All err within round
-----------------------------------
Step from index 6
All layers in step the unique
All epochs in step the same
All lr in step the same
All err within round
-----------------------------------
Step from index 9
All layers in step the unique
All epochs in step the same
All lr in step the same
All err within round
-----------------------------------
Step from index 12
All layers in step the unique
All epochs in step the same
All lr in step the same
All err within round
-----------------------------------
Step from index 15
All layers in step the unique
All epochs in step the same
All lr in step the same
{208, 209}
-----------------------------------
Step from index 18
All layers in step the unique
All epochs in ste

In [133]:
# TODO: test extremely simplified network structures see beneath as well
# TODO: test an extremely complexe network structure (just a lot of layers)
# TODO: test stable network for whole range of epochs and record results
# TODO: test stable network performance on hard lim transfer functions
# TODO: test stable network performance but training it with a hardlim transfer function last layer
# TODO: redo all this with symetrical network

In [113]:
# Test performance on an extremely simple network structure
# Network parameters
R = 4
layer_sizes = [4, 8, 2]
deriv_transfer_functions = [deriv_logsig, deriv_lin]
# Inputs and outputs
inputs, outputs, S = gen_inputs_outputs(R)
layer_sizes = np.array(layer_sizes + [S], dtype=int)

In [114]:
reset_params()
train_network(0.001953125, epochs=12)
print(performance())

IndexError: list index out of range

In [110]:
# Test performance on an extremely complexe network structure
# Network parameters
R = 4
layer_sizes = [4, 1024, 1024, 1024, 1024]
transfer_functions = [logsig, logsig, logsig, logsig, lin]
deriv_transfer_functions = [deriv_logsig, deriv_logsig, deriv_logsig, deriv_logsig, deriv_lin]
# Inputs and outputs
inputs, outputs, S = gen_inputs_outputs(R)
layer_sizes = np.array(layer_sizes + [S], dtype=int)

In [111]:
reset_params()
train_network(0.001953125, epochs=12)
print(performance())

3.2955718257616284e+190


In [13]:
# Test the model with keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD

In [14]:
model = Sequential()
model.add(Dense(layer_sizes[1], input_shape=(layer_sizes[0],), activation='sigmoid'))
model.add(Dense(layer_sizes[2], activation='linear'))

In [15]:
sdg = SGD(.9)
model.compile(loss="categorical_crossentropy", optimizer=sdg,
	metrics=["accuracy"])
model.fit(inputs, outputs)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step - accuracy: 0.5000 - loss: 7.0425


In [16]:
model.predict(inputs)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step


array([[ 0.01186371, -8.282264  , -1.4656483 ],
       [ 0.0887537 , -9.469966  , -1.5549538 ],
       [ 0.15937938, -8.323184  , -1.3819622 ],
       [ 0.22767666, -9.4916    , -1.4718292 ],
       [-0.01628557, -8.611647  , -1.6199257 ],
       [ 0.04203045, -9.72632   , -1.6907108 ],
       [ 0.14307892, -8.662352  , -1.5352875 ],
       [ 0.20412186, -9.765932  , -1.603532  ]], dtype=float32)